## System Prompt for DALLE-Whisperer

You are an assistant tasked with generating highly detailed and consistent prompts for DALL-E 3 to create visually cohesive storyboards.

Follow these instructions for every prompt:
1. **Core Character Design**:
   - Describe each character's physical appearance, including unique traits like face, hair, or build.
   - Include any defining features (e.g., clothing, accessories) to ensure character consistency across images.

2. **Style Attributes**:
   - Use "digital painting" as the base style.
   - Include style attributes like gradient shading, clean linework, vibrant palette, and stylized proportions.

3. **Scene Context**:
   - For every scene, provide:
     - **A Setting**: Describe the background environment.
     - **An Action**: Specify dynamic actions the character(s) are performing.
     - **Emotion**: Highlight a strong emotion expressed by the character(s).

4. **Output Format**:
   - Always use the following template:
     ```
     Digital painting of [character appearance] with [style attributes]. Wearing [clothing/accessories], [scenario: setting, action, emotion].
     ```

Keep the character appearance consistent across different scenarios to ensure visual coherence in the storyboard.


In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

load_dotenv()

True

## DALL-E Whisperer Inputs (Per Scene)

#### Scene Context
For each scene, provide the following:
- **Characters**: A description of the characters involved.
- **Action**: What are the characters doing?
- **Emotion**: What emotion(s) are they expressing?
- **Setting**: Where is the scene taking place?


In [ ]:
import re

# Predefined character and settings dictionaries from existing analysis
existing_characters = {
    "mr_summers": "a round-faced, jovial man, wearing a formal jacket and holding a black wooden box",
    "bobby_martin": "a mischievous boy with tousled hair, wearing simple overalls",
    "father": "a stern man with a weathered face, wearing a farmer's hat and plain shirt"
}

existing_settings = {
    "village_square": "a bustling square between the post office and the bank, with green grass and blooming flowers"
}

# Function to cross-reference and complete the shot descriptions
def cross_reference_shots(shots, characters, settings):
    """
    Cross-references shot descriptions with existing characters and settings.
    Adds temporary descriptions for missing elements.
    
    Args:
        shots (list): List of shot descriptions.
        characters (dict): Existing character dictionary.
        settings (dict): Existing settings dictionary.
    
    Returns:
        list: Updated shot descriptions with cross-referenced and completed elements.
    """
    updated_shots = []

    for shot in shots:
        # Extract setting
        setting_match = re.search(r"{setting: (.*?)}", shot)
        setting_name = setting_match.group(1) if setting_match else None
        setting_description = settings.get(setting_name, f"a generic depiction of {setting_name}")
        
        # Extract characters
        character_matches = re.findall(r"{character: (.*?)}", shot)
        character_descriptions = []
        for char in character_matches:
            if char in characters:
                character_descriptions.append(characters[char])
            else:
                # Generate a temporary description for extras
                temp_description = f"an unnamed extra with nondescript attire and neutral expression ({char})"
                characters[char] = temp_description  # Add to characters for consistency
                character_descriptions.append(temp_description)
        
        # Rebuild the shot description with detailed cross-references
        detailed_shot = f"Setting: {setting_description}\n"
        detailed_shot += f"Characters: {', '.join(character_descriptions)}\n"
        detailed_shot += shot  # Retain the original shot for context
        
        updated_shots.append(detailed_shot)

    return updated_shots

# Example shot descriptions
shots = [
    "{setting: village_square} A wide-angle shot of the villagers gathering in the square. {character: mr_summers} arrives, carrying a black wooden box.",
    "{setting: village_square} {character: woman_1} chats with {character: man_1}, laughing as they exchange gossip.",
    "{setting: village_square} {character: bobby_martin} ducks under his {character: mother}'s grasp, running to a pile of stones.",
]

# Cross-reference and complete the shots
completed_shots = cross_reference_shots(shots, existing_characters, existing_settings)

# Display the completed shot descriptions
for shot in completed_shots:
    print(shot)
    print("-" * 50)


In [3]:
# Set up OpenAI client
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")  # Ensure your API key is set as an environment variable
)

def generate_dalle_prompts(characters, settings, script):
    """
    Generate a list of DALL-E prompts based on characters, settings, and script.
    :param characters: List of character descriptions.
    :param settings: List of settings or scene environments.
    :param script: List of scenes or actions.
    :return: List of DALL-E prompts.
    """
    system_prompt = """
    You are an assistant tasked with generating consistent DALL-E prompts for a storyboard. 
    Follow these rules for each prompt:
    - Use the "digital painting" style with gradient shading, clean linework, vibrant palette, and stylized proportions.
    - Maintain character consistency in appearance, clothing, and accessories.
    - Each prompt should include:
      1. A detailed description of the character(s) in the scene.
      2. A dynamic action the character(s) is performing.
      3. The setting where the scene takes place.
      4. The emotion being expressed by the character(s).
    Format each prompt as:
    "Digital painting of [character appearance] with [style attributes]. Wearing [clothing/accessories], [scenario: setting, action, emotion]."
    """
    
    # User input structured based on script, characters, and settings
    user_prompt = "Generate DALL-E prompts for the following scenes:\n"
    for i, (character, setting, scene) in enumerate(zip(characters, settings, script)):
        user_prompt += f"\nScene {i+1}:\n- Characters: {character}\n- Setting: {setting}\n- Action/Scene: {scene}\n"
    
    # Create chat completion
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        model="gpt-4o",
    )
    
    # Extract and return prompts
    response = chat_completion["choices"][0]["message"]["content"]
    return response.strip().split("\n")
